In [44]:
import os
import collections
import numpy as np
import pandas as pd
from time import time

import plotly.graph_objects as go
from plotly.subplots import make_subplots

data_dir = "/home/stavros/DATA/TripAdvisorReviews"

In [14]:
hotel_dir = os.path.join(data_dir, "kresten_royal/the_kresten_royal_villas_1747reviews_withaspects")
data = containers.DataAspects.load(data_dir=hotel_dir)

In [74]:
import gensim
google_vec_file = os.path.join("/home/stavros/DATA/GoogleNews-vectors-negative300.bin.gz")
word2vec = gensim.models.KeyedVectors.load_word2vec_format(google_vec_file, binary=True, limit=200000)

categories = ["location", "cleanliness", "service", "value"]
# Verify that all categories are in model
for cat in categories:
    print(cat in word2vec)

True
True
True
True


In [83]:
any([False, False, False])

False

In [79]:
def calculate_distances(phrase: str):
    words = phrase.split(" ")
    d = np.stack([word2vec.distances(word, categories) for word in words if word in word2vec])
    return d.mean(axis=0)

In [85]:
ordered_phrases = []
for phrase in data.container.words:
    if any([word in word2vec for word in phrase.split(" ")]):
        ordered_phrases.append(phrase)

print(len(ordered_phrases), len(data.container.words))
print(len(ordered_phrases) / len(data.container.words))

2490 2719
0.9157778595071717


In [86]:
categorical_distances = np.array([calculate_distances(phrase) for phrase in ordered_phrases])
categorical_distances.shape

(2490, 4)

In [105]:
categorical_distances

array([[0.95680815, 0.950755  , 0.88873446, 0.96747005],
       [0.9588536 , 0.8752915 , 0.9706222 , 1.0261949 ],
       [0.9351849 , 0.8793582 , 0.9378916 , 0.909578  ],
       ...,
       [0.9649677 , 0.9966132 , 0.9006722 , 0.88993   ],
       [0.8867016 , 0.9520921 , 0.95121646, 1.0417291 ],
       [0.8947793 , 1.0141004 , 0.92664135, 0.89449483]], dtype=float32)

In [87]:
np.unique(categorical_distances.argmin(axis=-1), return_counts=True)

(array([0, 1, 2, 3]), array([677, 817, 570, 426]))

In [95]:
phrase2cat = {phrase: (categories[dist.argmin()], dist.min()) 
              for phrase, dist in zip(ordered_phrases, categorical_distances)}
print(len(phrase2cat))

2490


In [103]:
groups_appearances = {cat: collections.Counter({}) for cat in categories}
groups_distance = {cat: collections.Counter({}) for cat in categories}
for phrase, (cat, dist) in phrase2cat.items():
    groups_appearances[cat][phrase] = data.container.appearances[phrase]
    groups_distance[cat][phrase] = 1 - dist

In [92]:
for k, group in groups.items():
    print(k, "location" in group)

location True
cleanliness False
service False
value False


In [104]:
groups_distance["location"].most_common()

[('location', 1.0),
 ('-excellent location', 1.0),
 ('locations', 0.6393399238586426),
 ('’s hotel location', 0.6237652897834778),
 ('hotel location', 0.6237652897834778),
 ('proximity', 0.6181362271308899),
 ('hotels location', 0.6168566346168518),
 ('sea location', 0.5363045334815979),
 ('destination', 0.5136812925338745),
 ('venue', 0.48242413997650146),
 ('area', 0.46242427825927734),
 ('-pool area', 0.46242427825927734),
 ('facility', 0.39675372838974),
 ('restaurant area', 0.3944557309150696),
 ('site complex', 0.3602726459503174),
 ('hotel area', 0.35497742891311646),
 ('layout', 0.35242414474487305),
 ('surroundings', 0.34995579719543457),
 ('store', 0.3488495349884033),
 ('places', 0.34859025478363037),
 ('destinations', 0.34319615364074707),
 ('timing', 0.3387160897254944),
 ('spot', 0.33309078216552734),
 ('ambiance', 0.329852819442749),
 ('dining area', 0.3284781575202942),
 ('directions', 0.3279449939727783),
 ('place', 0.32734042406082153),
 ('locality', 0.327026247978210

In [94]:
for k, group in groups.items():
    print(k)
    print(group.most_common(30))
    print("\n\n")

location
[('hotel', 529), ('room', 209), ('view', 189), ('time', 176), ('pool', 140), ('place', 139), ('location', 117), ('stay', 114), ('choice', 113), ('selection', 93), ('week', 92), ('thing', 63), ('bar', 59), ('pool areas', 51), ('area', 50), ('one', 49), ('day', 47), ('atmosphere', 46), ('size', 41), ('restaurant', 40), ('days', 38), ('facilities', 38), ('way', 37), ('star', 35), ('weather', 32), ('night', 31), ('start', 29), ('areas', 28), ('town', 28), ('trip', 27)]



cleanliness
[('food', 297), ('rooms', 117), ('quality', 112), ('pools', 88), ('reviews', 86), ('drinks', 83), ('water', 71), ('towels', 64), ('beds', 56), ('beach', 56), ('restaurants', 37), ('things', 36), ('drink', 33), ('eater', 30), ('hotels', 28), ('dishes', 27), ('smell', 26), ('bathroom', 25), ('clean', 23), ('bed', 23), ('guests', 20), ('sunbeds', 19), ('hotel staff', 17), ('eaters', 17), ('air conditioning', 16), ('nights', 16), ('plates', 16), ('lady', 16), ('bars', 14), ('tables', 14)]



service
[('st